In [36]:
import sys

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sys.path.append("./utils")

In [37]:
import loader
import round_classifier
import thresholds
from Parser import Parser

In [38]:
all_data = loader.load_dataset()

In [39]:
all_columns = all_data.columns
all_columns[0:16]

Index(['time_left', 'ct_score', 't_score', 'map', 'bomb_planted', 'ct_health',
       't_health', 'ct_armor', 't_armor', 'ct_money', 't_money', 'ct_helmets',
       't_helmets', 'ct_defuse_kits', 'ct_players_alive', 't_players_alive'],
      dtype='object')

In [40]:
parser = Parser()
# treat_data = all_data.copy()
treat_data = parser.classify_weapons(all_data.copy())
# treat_data = parser.create_round_winner_columns(treat_data.copy())

In [41]:
treat_data["pistol_round"] = treat_data.apply(
    round_classifier.define_pistol_round, axis=1
)

# Defining eco rounds
treat_data["ct_eco"] = treat_data.apply(
    round_classifier.define_full_ct_eco, axis=1)
treat_data["t_eco"] = treat_data.apply(
    round_classifier.define_full_t_eco, axis=1)

# Defining force buy rounds
treat_data["ct_force"] = treat_data.apply(
    round_classifier.define_force_ct_round, axis=1
)
treat_data["t_force"] = treat_data.apply(
    round_classifier.define_force_t_round, axis=1)


In [42]:
# Selecting columns
normal_columns = all_columns[0:16].tolist()
made_columns_t = [
    "t_main_rifle",
    "t_sec_rifle",
    "t_force_weapons",
    "t_weak_pistols",
    "t_strong_pistols",
    "t_granades",
    "t_weapon_awp",
    "t_eco",
    "t_force",
]
made_columns_ct = [
    "ct_main_rifle",
    "ct_sec_rifle",
    "ct_force_weapons",
    "ct_weak_pistols",
    "ct_strong_pistols",
    "ct_granades",
    "ct_weapon_awp",
    "ct_eco",
    "ct_force",
]
extra_columns = [
    "pistol_round",
    "round_winner",
    # "round_winner_t",
    # "round_winner_ct",
    # "t_health_thresholds",
    # "t_health_thresholds_num",
    # "ct_health_thresholds",
    # "ct_health_thresholds_num",
]
selected_columns = normal_columns + made_columns_ct + made_columns_t + extra_columns

# Getting only columns selected columns
final_df = treat_data[selected_columns]
# checando nans
final_df

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_sec_rifle,t_force_weapons,t_weak_pistols,t_strong_pistols,t_granades,t_weapon_awp,t_eco,t_force,pistol_round,round_winner
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,CT
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0,0,1,CT
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0,0,1,CT
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,CT
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,15.41,11.0,14.0,de_train,True,200.0,242.0,195.0,359.0,100.0,...,0.0,0.0,2.0,0.0,3.0,1.0,0,0,0,T
122406,174.93,11.0,15.0,de_train,False,500.0,500.0,95.0,175.0,11500.0,...,0.0,0.0,0.0,0.0,2.0,1.0,0,0,0,T
122407,114.93,11.0,15.0,de_train,False,500.0,500.0,495.0,475.0,1200.0,...,0.0,0.0,0.0,1.0,14.0,1.0,0,0,0,T
122408,94.93,11.0,15.0,de_train,False,500.0,500.0,495.0,475.0,1200.0,...,0.0,0.0,0.0,1.0,12.0,1.0,0,0,0,T


In [43]:
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
from sklearn.model_selection import train_test_split
seed = 42
y = final_df.pop('round_winner')
X = final_df
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.333, random_state = seed)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.333, random_state = seed)
features_to_encode = ["map","bomb_planted","ct_armor","t_armor","ct_helmets","t_helmets","ct_defuse_kits","ct_eco","ct_force","t_eco","t_force","pistol_round"]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
col_trans = make_column_transformer(
                        (OneHotEncoder(),features_to_encode),
                        remainder = "passthrough")
col_trans

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(
                      min_samples_leaf=50,
                      n_estimators=150,
                      bootstrap=True,
                      oob_score=True,
                      n_jobs=-1,
                      random_state=seed,
                      max_features='auto')

In [ ]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(col_trans, rf_classifier)
pipe.fit(X_train, y_train)

In [ ]:
y_pred = pipe.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, f1_score
accuracy_score(y_test, y_pred)


In [ ]:
# from sklearn.metrics import plot_confusion_matrix

# plot_confusion_matrix(rf_classifier,y_test, y_pred)